## Lesson 1 - Can we just generate blind synonyms?

In this lesson we work on generating blind synonyms using LLMs. This is often the first stop for search teams.

To run:
* Either make sure Ollama is running or you have an `OPENAI_API_KEY` installed

In [1]:
from cheat_at_search.wands_data import products, queries
from cheat_at_search.strategy import (
    BM25Search,
    MiniLMSearch,
    EnrichedBM25Search,
    EnrichedJustRoomBM25Search,
    SynonymSearch,
)

bm25 = BM25Search(products)
synonym = SynonymSearch(products)

2025-05-28 11:10:09,292 - cheat_at_search.wands_data - INFO - Directory /Users/doug/ws/cheat-with-llms/notebooks/data/wands already exists. Skipping clone.
2025-05-28 11:10:09,293 - cheat_at_search.wands_data - INFO - Loading relevance labels from /Users/doug/ws/cheat-with-llms/notebooks/data/wands/dataset/label.csv
2025-05-28 11:10:09,356 - cheat_at_search.wands_data - INFO - Loaded 231873 relevance labels
2025-05-28 11:10:09,356 - cheat_at_search.wands_data - INFO - Directory /Users/doug/ws/cheat-with-llms/notebooks/data/wands already exists. Skipping clone.
2025-05-28 11:10:09,356 - cheat_at_search.wands_data - INFO - Loading queries from /Users/doug/ws/cheat-with-llms/notebooks/data/wands/dataset/query.csv
2025-05-28 11:10:09,357 - cheat_at_search.wands_data - INFO - Loaded 480 queries
2025-05-28 11:10:09,358 - cheat_at_search.wands_data - INFO - Directory /Users/doug/ws/cheat-with-llms/notebooks/data/wands already exists. Skipping clone.
2025-05-28 11:10:09,358 - cheat_at_search.w

/Users/doug/ws/cheat-with-llms/cheat_at_search/wands_data.py:83: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['product_description'].fillna('', inplace=True)
/Users/doug/ws/cheat-with-llms/cheat_at_search/wands_data.py:84: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

2025-05-28 11:10:12,362 - searcharray.indexing - INFO - Indexing begins w/ 4 workers
2025-05-28 11:10:12,364 - searcharray.indexing - INFO - 0 Batch Start tokenization
2025-05-28 11:10:12,364 - searcharray.indexing - INFO - Tokenizing 42994 documents
2025-05-28 11:10:12,470 - searcharray.indexing - INFO - Tokenized 10000 (23.259059403637718%)
2025-05-28 11:10:12,570 - searcharray.indexing - INFO - Tokenized 20000 (46.518118807275435%)
2025-05-28 11:10:12,663 - searcharray.indexing - INFO - Tokenized 30000 (69.77717821091315%)
2025-05-28 11:10:12,754 - searcharray.indexing - INFO - Tokenized 40000 (93.03623761455087%)
2025-05-28 11:10:12,798 - searcharray.indexing - INFO - Tokenization -- vstacking
2025-05-28 11:10:12,799 - searcharray.indexing - INFO - Tokenization -- DONE
2025-05-28 11:10:12,801 - searcharray.indexing - INFO - Inverting docs->terms
2025-05-28 11:10:12,819 - searcharray.indexing - INFO - Encoding positions to bit array
2025-05-28 11:10:12,832 - searcharray.indexing - I

### Ideal top 10 for each query

In [2]:
from cheat_at_search.wands_data import labeled_queries

def ideal10():
    
    ideal_results = labeled_queries.sort_values(['query_id', 'grade'], ascending=(True, False))
    ideal_results['rank'] = ideal_results.groupby('query_id').cumcount() + 1
    ideal_top_10 = ideal_results[ideal_results['rank'] <= 10] \
        .add_prefix('ideal_') \
        .rename(columns={'ideal_query_id': 'query_id', 'ideal_query': 'query'})
    
    ideal_top_10 = ideal_top_10.merge(
        products[['product_id', 'product_name']], how='left', left_on='ideal_product_id', right_on='product_id'
    ).rename(columns={'product_name': 'ideal_product_name'}).drop(columns='ideal_query_class')
    
    return ideal_top_10

ideal_top_10 = ideal10()
ideal_top_10

,query_id,query,ideal_product_id,ideal_id,ideal_label,ideal_grade,ideal_rank,product_id,ideal_product_name
0,0,salon chair,1197,99,Exact,2.0,1,1197,lizzy 25.6 '' w bariatric antimicrobial waitin...
1,0,salon chair,1198,116,Exact,2.0,2,1198,rackham 26.8 '' w bariatric antimicrobial pvc ...
2,0,salon chair,2636,3,Exact,2.0,3,2636,25 '' wide faux leather manual swivel standard...
3,0,salon chair,2638,66,Exact,2.0,4,2638,faux leather massage chair
4,0,salon chair,5936,10,Exact,2.0,5,5936,69 '' w metal seat waiting room chair with met...
...,...,...,...,...,...,...,...,...,...
4732,487,rack glass,110,202050,Partial,1.0,6,110,west harptree 34 '' w garment rack
4733,487,rack glass,285,202336,Partial,1.0,7,285,3 shelf movable storage rack
4734,487,rack glass,573,202462,Partial,1.0,8,573,5 - hook freestanding coat rack
4735,487,rack glass,635,202178,Partial,1.0,9,635,4 - hook freestanding coat rack


## Run a simple BM25 search, evaluate

* Simple BM25 search, `/cheat_at_search/strategies/bm25.py`

In [3]:
from cheat_at_search.eval import grade_results

def run_strategy(strategy):
    
    results = strategy.search_all(queries)
    graded = grade_results(
        results,
        max_grade=2,
        k=10,
    )
    
    idcg = graded['idcg'].iloc[0]
    dcgs = graded.groupby(["query", 'query_id'])["discounted_gain"].sum().sort_values(ascending=False).rename('dcg')
    ndcgs = dcgs / idcg
    ndcgs = ndcgs.rename('ndcg')
    
    graded = graded.merge(dcgs, on=['query', 'query_id'])
    graded = graded.merge(ndcgs, on=['query', 'query_id'])
    
    return graded

graded_bm25 = run_strategy(bm25)
graded_bm25

/Users/doug/ws/cheat-with-llms/cheat_at_search/eval.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  graded_results['grade'].fillna(0, inplace=True)


,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count,features,...,query_id,rank,query_class,id,label,grade,discounted_gain,idcg,dcg,ndcg
0,7465,hair salon chair,Massage Chairs|Recliners,Furniture / Living Room Furniture / Chairs & S...,offers a wide selection of professional salon ...,fauxleathertype : pu|legheight-toptobottom:18|...,69.0,4.5,53.0,"[fauxleathertype : pu, legheight-toptobottom:1...",...,0,1,Massage Chairs,80.0,Exact,2.0,3.00,8.786905,8.536905,0.971549
1,7468,mercer41 hair salon chair hydraulic styling ch...,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,mercer41 beauty offers a wide selection profes...,seatfillmaterial : foam|waterrepellant : no re...,1.0,5.0,1.0,"[seatfillmaterial : foam, waterrepellant : no ...",...,0,2,Massage Chairs,104.0,Exact,2.0,1.50,8.786905,8.536905,0.971549
2,25431,barberpub salon massage chair,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,salon chairs are a wonderful avenue for hairst...,supplierintendedandapproveduse : non residenti...,4.0,5.0,4.0,[supplierintendedandapproveduse : non resident...,...,0,3,Massage Chairs,29.0,Exact,2.0,1.00,8.786905,8.536905,0.971549
3,25432,barberpub hydraulic salon spa reclining massag...,Massage Chairs|Recliners,Furniture / Living Room Furniture / Chairs & S...,salon chairs are a wonderful avenue for hairst...,backheight-seattotopofback:15.7|recliningtyped...,5.0,4.0,5.0,"[backheight-seattotopofback:15.7, recliningtyp...",...,0,4,Massage Chairs,28.0,Exact,2.0,0.75,8.786905,8.536905,0.971549
4,15612,massage chair,Massage Chairs,Furniture / Living Room Furniture / Chairs & S...,features heavy duty steel frame . premium chro...,overallheight-toptobottom:35.5|productcare : d...,59.0,4.5,50.0,"[overallheight-toptobottom:35.5, productcare :...",...,0,5,Massage Chairs,101.0,Exact,2.0,0.60,8.786905,8.536905,0.971549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,40243,madisen hanging wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,complement your farmhouse kitchen decor with t...,producttype : wine glass rack|overallwidth-sid...,29.0,5.0,20.0,"[producttype : wine glass rack, overallwidth-s...",...,487,6,NaN,NaN,NaN,0.0,0.00,8.786905,0.000000,0.000000
4796,40244,kena hanging wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,spruce up your farmhouse kitchen decor with th...,warrantylength:1 year|producttype : wine glass...,23.0,5.0,18.0,"[warrantylength:1 year, producttype : wine gla...",...,487,7,NaN,NaN,NaN,0.0,0.00,8.786905,0.000000,0.000000
4797,39976,wall mounted wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,"the latest addition to this collection , this ...",overallheight-toptobottom:4|design : wall moun...,34.0,4.5,18.0,"[overallheight-toptobottom:4, design : wall mo...",...,487,8,NaN,NaN,NaN,0.0,0.00,8.786905,0.000000,0.000000
4798,40247,winn hanging wine glass rack,Wine Racks,Kitchen & Tabletop / Tableware & Drinkware / B...,are you looking for a safe and decorative solu...,overallheight-toptobottom:1.5|overallwidth-sid...,305.0,5.0,187.0,"[overallheight-toptobottom:1.5, overallwidth-s...",...,487,9,NaN,NaN,NaN,0.0,0.00,8.786905,0.000000,0.000000


In [13]:
def side_by_side(graded, ideal_top_10):
    
    graded_view = graded[['query_id', 'query', 'rank', 'product_id', 'product_name',  'grade', 'dcg', 'ndcg']].rename(
                          columns={'product_id': 'product_id_actual', 'product_name': 'product_name_actual'})
    
    sxs = ideal_top_10.merge(graded_view, 
                             how='left',
                             left_on=['query_id', 'query', 'ideal_rank'],
                             right_on=['query_id', 'query', 'rank'])
    return sxs


sxs_bm25 = side_by_side(graded_bm25, ideal_top_10)
sxs_bm25

,query_id,query,ideal_product_id,ideal_id,ideal_label,ideal_grade,ideal_rank,product_id,ideal_product_name,rank,product_id_actual,product_name_actual,grade,dcg,ndcg
0,0,salon chair,1197,99,Exact,2.0,1,1197,lizzy 25.6 '' w bariatric antimicrobial waitin...,1,7465,hair salon chair,2.0,8.536905,0.971549
1,0,salon chair,1198,116,Exact,2.0,2,1198,rackham 26.8 '' w bariatric antimicrobial pvc ...,2,7468,mercer41 hair salon chair hydraulic styling ch...,2.0,8.536905,0.971549
2,0,salon chair,2636,3,Exact,2.0,3,2636,25 '' wide faux leather manual swivel standard...,3,25431,barberpub salon massage chair,2.0,8.536905,0.971549
3,0,salon chair,2638,66,Exact,2.0,4,2638,faux leather massage chair,4,25432,barberpub hydraulic salon spa reclining massag...,2.0,8.536905,0.971549
4,0,salon chair,5936,10,Exact,2.0,5,5936,69 '' w metal seat waiting room chair with met...,5,15612,massage chair,2.0,8.536905,0.971549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4732,487,rack glass,110,202050,Partial,1.0,6,110,west harptree 34 '' w garment rack,6,40243,madisen hanging wine glass rack,0.0,0.000000,0.000000
4733,487,rack glass,285,202336,Partial,1.0,7,285,3 shelf movable storage rack,7,40244,kena hanging wine glass rack,0.0,0.000000,0.000000
4734,487,rack glass,573,202462,Partial,1.0,8,573,5 - hook freestanding coat rack,8,39976,wall mounted wine glass rack,0.0,0.000000,0.000000
4735,487,rack glass,635,202178,Partial,1.0,9,635,4 - hook freestanding coat rack,9,40247,winn hanging wine glass rack,0.0,0.000000,0.000000


In [14]:
sxs_bm25[sxs_bm25['query'] == 'ottoman bed queen']

,query_id,query,ideal_product_id,ideal_id,ideal_label,ideal_grade,ideal_rank,product_id,ideal_product_name,rank,product_id_actual,product_name_actual,grade,dcg,ndcg
3381,344,ottoman bed queen,545,37893,Partial,1.0,1,545,loggins storage ottoman,1,2089,elzadie queen four poster bed,0.0,0.0,0.0
3382,344,ottoman bed queen,998,37883,Partial,1.0,2,998,karlsefni 37.5 '' wide tufted rectangle standa...,2,38301,sinead queen upholstered platform bed,0.0,0.0,0.0
3383,344,ottoman bed queen,1030,37836,Partial,1.0,3,1030,48 '' tufted rectangle standard ottoman,3,14834,abbot diamond queen upholstered standard bed,0.0,0.0,0.0
3384,344,ottoman bed queen,1367,37901,Partial,1.0,4,1367,roft 50 '' tufted rectangle storage ottoman,4,7371,carrollton queen upholstered standard bed,0.0,0.0,0.0
3385,344,ottoman bed queen,1537,37873,Partial,1.0,5,1537,henninger 25 '' tufted round storage ottoman,5,29090,lamberton queen upholstered platform bed,0.0,0.0,0.0
3386,344,ottoman bed queen,1563,37894,Partial,1.0,6,1563,luper 41.75 '' wide tufted rectangle storage o...,6,17289,masonville queen platform bed,0.0,0.0,0.0
3387,344,ottoman bed queen,1730,37910,Partial,1.0,7,1730,tufted storage ottoman,7,17219,bianaca queen low profile four poster bed,0.0,0.0,0.0
3388,344,ottoman bed queen,1753,37854,Partial,1.0,8,1753,camden 28.5 '' wide square folding bed cocktai...,8,6503,karmakar queen upholstered platform bed,0.0,0.0,0.0
3389,344,ottoman bed queen,1844,37889,Partial,1.0,9,1844,lampkins 24 '' rectangle standard ottoman,9,34268,fabienne queen standard bed,0.0,0.0,0.0
3390,344,ottoman bed queen,1927,37864,Partial,1.0,10,1927,denali 51.18 '' wide faux leather rectangle st...,10,32412,louque queen upholstered platform bed,0.0,0.0,0.0


## Lets try a simple synonym search

Lets inspect SynonymSearch in strategies to see what it does, then lets run it to compare.

In [7]:
graded_syn = run_strategy(synonym)
sxs_syn = side_by_side(graded_syn, ideal_top_10)
sxs_syn

/Users/doug/ws/cheat-with-llms/cheat_at_search/eval.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  graded_results['grade'].fillna(0, inplace=True)


,query_id,query,ideal_product_id,ideal_id,ideal_label,ideal_grade,ideal_rank,product_id,ideal_product_name,rank,product_id_actual,product_name_actual,grade,dcg,ndcg
0,0,salon chair,1197,99,Exact,2.0,1,1197,lizzy 25.6 '' w bariatric antimicrobial waitin...,1,36868,dockrey wall mount salon spa hair styling stat...,0.0,5.564683,0.633293
1,0,salon chair,1198,116,Exact,2.0,2,1198,rackham 26.8 '' w bariatric antimicrobial pvc ...,2,7465,hair salon chair,2.0,5.564683,0.633293
2,0,salon chair,2636,3,Exact,2.0,3,2636,25 '' wide faux leather manual swivel standard...,3,7468,mercer41 hair salon chair hydraulic styling ch...,2.0,5.564683,0.633293
3,0,salon chair,2638,66,Exact,2.0,4,2638,faux leather massage chair,4,25431,barberpub salon massage chair,2.0,5.564683,0.633293
4,0,salon chair,5936,10,Exact,2.0,5,5936,69 '' w metal seat waiting room chair with met...,5,25432,barberpub hydraulic salon spa reclining massag...,2.0,5.564683,0.633293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4732,487,rack glass,110,202050,Partial,1.0,6,110,west harptree 34 '' w garment rack,6,39976,wall mounted wine glass rack,0.0,0.142857,0.016258
4733,487,rack glass,285,202336,Partial,1.0,7,285,3 shelf movable storage rack,7,34600,croll 12 bottle floor wine bottle and glass rack,1.0,0.142857,0.016258
4734,487,rack glass,573,202462,Partial,1.0,8,573,5 - hook freestanding coat rack,8,40243,madisen hanging wine glass rack,0.0,0.142857,0.016258
4735,487,rack glass,635,202178,Partial,1.0,9,635,4 - hook freestanding coat rack,9,40244,kena hanging wine glass rack,0.0,0.142857,0.016258


In [10]:
ndcg_syn = graded_syn.groupby('query')['ndcg'].first().sort_values()
ndcg_bm25 = graded_bm25.groupby('query')['ndcg'].first().sort_values()
(ndcg_syn - ndcg_bm25).sort_values()

query
chair pillow cushion      -1.000000
dining room tables        -0.962065
marble                    -0.923181
cortez pillow             -0.886194
amarillo                  -0.885246
                             ...   
leather chair              0.366120
floating bed               0.381565
tye dye duvet cover        0.442217
kitchen storage cabinet    0.463623
drum picture               0.512126
Name: ndcg, Length: 480, dtype: float64

In [11]:
sxs_syn[sxs_syn['query'] == 'chair pillow cushion']

,query_id,query,ideal_product_id,ideal_id,ideal_label,ideal_grade,ideal_rank,product_id,ideal_product_name,rank,product_id_actual,product_name_actual,grade,dcg,ndcg
4604,473,chair pillow cushion,198,49537,Exact,2.0,1,198,indoor chair cushion,1,19968,home office chair with ergonomic adjustable hi...,0.0,0.0,0.0
4605,473,chair pillow cushion,200,58110,Exact,2.0,2,200,chair outdoor seat cushion,2,5542,indoor seat cushion,0.0,0.0,0.0
4606,473,chair pillow cushion,202,49525,Exact,2.0,3,202,chair indoor seat cushion,3,10294,deep outdoor seat cushion,0.0,0.0,0.0
4607,473,chair pillow cushion,570,49538,Exact,2.0,4,570,indoor dining chair cushion,4,39975,ergonomic memory foam seat cushion,0.0,0.0,0.0
4608,473,chair pillow cushion,571,49540,Exact,2.0,5,571,indoor rocking chair cushion,5,9648,outdoor seat cushion,0.0,0.0,0.0
4609,473,chair pillow cushion,1372,49524,Exact,2.0,6,1372,chair cushion,6,27635,ergonomic gel seat cushion,0.0,0.0,0.0
4610,473,chair pillow cushion,1376,58105,Exact,2.0,7,1376,rockhill indoor/outdoor chair cushion,7,20037,home office decor indoor/outdoor seat cushion,0.0,0.0,0.0
4611,473,chair pillow cushion,1828,58124,Exact,2.0,8,1828,indoor/outdoor dining chair cushion,8,20124,indoor/outdoor seat cushion,0.0,0.0,0.0
4612,473,chair pillow cushion,1830,197746,Exact,2.0,9,1830,solid indoor/outdoor lounge chair cushion,9,8692,patio d indoor/outdoor seat cushion,0.0,0.0,0.0
4613,473,chair pillow cushion,1881,49571,Exact,2.0,10,1881,sarver indoor/outdoor seat/back cushion,10,20039,home office use soft flax outdoor seat cushion,0.0,0.0,0.0


In [15]:
sxs_bm25[sxs_bm25['query'] == 'chair pillow cushion']

,query_id,query,ideal_product_id,ideal_id,ideal_label,ideal_grade,ideal_rank,product_id,ideal_product_name,rank,product_id_actual,product_name_actual,grade,dcg,ndcg
4604,473,chair pillow cushion,198,49537,Exact,2.0,1,198,indoor chair cushion,1,30279,replacement pillows outdoor lounge chair cushion,2.0,8.786905,1.0
4605,473,chair pillow cushion,200,58110,Exact,2.0,2,200,chair outdoor seat cushion,2,37578,indoor/outdoor dining chair cushion and pillow...,2.0,8.786905,1.0
4606,473,chair pillow cushion,202,49525,Exact,2.0,3,202,chair indoor seat cushion,3,7167,abbottsmoor dining chair cushion,2.0,8.786905,1.0
4607,473,chair pillow cushion,570,49538,Exact,2.0,4,570,indoor dining chair cushion,4,31137,zipparoll indoor chair cushion,2.0,8.786905,1.0
4608,473,chair pillow cushion,571,49540,Exact,2.0,5,571,indoor rocking chair cushion,5,198,indoor chair cushion,2.0,8.786905,1.0
4609,473,chair pillow cushion,1372,49524,Exact,2.0,6,1372,chair cushion,6,9636,tropical outdoor lounge chair cushion,2.0,8.786905,1.0
4610,473,chair pillow cushion,1376,58105,Exact,2.0,7,1376,rockhill indoor/outdoor chair cushion,7,3472,chair pad cushion,2.0,8.786905,1.0
4611,473,chair pillow cushion,1828,58124,Exact,2.0,8,1828,indoor/outdoor dining chair cushion,8,202,chair indoor seat cushion,2.0,8.786905,1.0
4612,473,chair pillow cushion,1830,197746,Exact,2.0,9,1830,solid indoor/outdoor lounge chair cushion,9,200,chair outdoor seat cushion,2.0,8.786905,1.0
4613,473,chair pillow cushion,1881,49571,Exact,2.0,10,1881,sarver indoor/outdoor seat/back cushion,10,9630,outdoor adirondack chair cushion,2.0,8.786905,1.0


In [16]:
synonym._synonyms('chair pillow cushion')

QueryWithSynonyms(keywords="[['chair', 'pillow', 'cushion']]", synonyms=[SynonymMapping(phrase='chair', synonyms=['office chair', 'stool', 'seating', 'sofa']), SynonymMapping(phrase='pillow', synonyms=['headrest', 'neck support', 'backrest', 'body pillow']), SynonymMapping(phrase='cushion', synonyms=['padded seat cover', 'seat cushion', 'armrest cushion', 'footrest'])])

In [17]:
synonym._synonyms('drum picture')

QueryWithSynonyms(keywords='drum picture', synonyms=[SynonymMapping(phrase='drum photo', synonyms=['drum image', 'drum artwork', 'drum photography']), SynonymMapping(phrase='guitar picture', synonyms=['guitar photo', 'guitar image', 'guitar photography']), SynonymMapping(phrase='music instrument picture', synonyms=['musical instrument', 'instrumentation', 'equipment']), SynonymMapping(phrase='wall art drum', synonyms=['drum wall decor', 'drum wall hanging', 'drum print'])])

In [18]:
sxs_bm25[sxs_bm25['query'] == 'drum picture']

,query_id,query,ideal_product_id,ideal_id,ideal_label,ideal_grade,ideal_rank,product_id,ideal_product_name,rank,product_id_actual,product_name_actual,grade,dcg,ndcg
1470,147,drum picture,6243,21573,Exact,2.0,1,6243,bourbon street blues iv by robert brasher - pi...,1,15321,lavada 4-light drum chandelier,0.0,0.0,0.0
1471,147,drum picture,6741,21596,Exact,2.0,2,6741,green drums graphic art on wrapped canvas,2,13904,jurgens mahogany picture frame,0.0,0.0,0.0
1472,147,drum picture,6742,21595,Exact,2.0,3,6742,green drums graphic art on plaque,3,29189,satrogan picture frame,0.0,0.0,0.0
1473,147,drum picture,11793,21584,Exact,2.0,4,11793,dog drumming painting,4,27494,gemmill matte picture frame,0.0,0.0,0.0
1474,147,drum picture,12093,21562,Exact,2.0,5,12093,2 piece drums picture of a drum kit percussion...,5,39625,design picture frame,0.0,0.0,0.0
1475,147,drum picture,12159,21633,Exact,2.0,6,12159,snare drum - wrapped canvas print,6,34840,elysium elephant picture frame,0.0,0.0,0.0
1476,147,drum picture,12186,21616,Exact,2.0,7,12186,multi-color drums on music sheet # 2 - graphic...,7,24581,denise 5 - light candle style drum chandelier,0.0,0.0,0.0
1477,147,drum picture,13874,21640,Exact,2.0,8,13874,the medicine drum by carl and grace moon photo...,8,29202,choe watercolor picture frame,0.0,0.0,0.0
1478,147,drum picture,15777,21587,Exact,2.0,9,15777,drummer boy - picture frame textual art print,9,16215,single picture frame,0.0,0.0,0.0
1479,147,drum picture,29890,21588,Exact,2.0,10,29890,drummers drumming by cathy walters - print,10,2428,ncaa university of alabama picture frame,0.0,0.0,0.0


In [19]:
sxs_syn[sxs_syn['query'] == 'drum picture']

,query_id,query,ideal_product_id,ideal_id,ideal_label,ideal_grade,ideal_rank,product_id,ideal_product_name,rank,product_id_actual,product_name_actual,grade,dcg,ndcg
1470,147,drum picture,6243,21573,Exact,2.0,1,6243,bourbon street blues iv by robert brasher - pi...,1,37178,percussion instruments chart,2.0,4.5,0.512126
1471,147,drum picture,6741,21596,Exact,2.0,2,6741,green drums graphic art on wrapped canvas,2,12093,2 piece drums picture of a drum kit percussion...,2.0,4.5,0.512126
1472,147,drum picture,6742,21595,Exact,2.0,3,6742,green drums graphic art on plaque,3,15321,lavada 4-light drum chandelier,0.0,4.5,0.512126
1473,147,drum picture,11793,21584,Exact,2.0,4,11793,dog drumming painting,4,13904,jurgens mahogany picture frame,0.0,4.5,0.512126
1474,147,drum picture,12093,21562,Exact,2.0,5,12093,2 piece drums picture of a drum kit percussion...,5,29189,satrogan picture frame,0.0,4.5,0.512126
1475,147,drum picture,12159,21633,Exact,2.0,6,12159,snare drum - wrapped canvas print,6,27494,gemmill matte picture frame,0.0,4.5,0.512126
1476,147,drum picture,12186,21616,Exact,2.0,7,12186,multi-color drums on music sheet # 2 - graphic...,7,39625,design picture frame,0.0,4.5,0.512126
1477,147,drum picture,13874,21640,Exact,2.0,8,13874,the medicine drum by carl and grace moon photo...,8,34840,elysium elephant picture frame,0.0,4.5,0.512126
1478,147,drum picture,15777,21587,Exact,2.0,9,15777,drummer boy - picture frame textual art print,9,24581,denise 5 - light candle style drum chandelier,0.0,4.5,0.512126
1479,147,drum picture,29890,21588,Exact,2.0,10,29890,drummers drumming by cathy walters - print,10,29202,choe watercolor picture frame,0.0,4.5,0.512126
